In [ ]:

import numpy as np
import matplotlib
import seaborn as sns
import pandas as pd
from pandas import DataFrame
#import mpl_toolkits
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from scipy.special import expit as sigmoid
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

In [ ]:
path = r'C:\Users\Nursultan\Documents\Data Science\breast cancer\wdbc.data.csv'
df = pd.read_csv(path, names = ['ID', 'Diagnosis',
                    'MRadius', 'MTexture', 'Mper-ter', 'Marea', 'MSmooth-es',
                    'MComp-ness', 'MConcavity', 'MConcavPoints', 'MSymm-ry',
                    'MFractDimens', 'RadiusSE', 'TextureSE', 'Per-terSE',
                    'AreaSE', 'Smooth-esSE',
                    'Comp-nessSE', 'ConcavitySE', 'ConcavPointsSE', 
                    'Symm-rySE', 'FractDimensSE', 'WRadius', 'WTexture',
                    'WPer-ter', 'WArea', 'WSmooth-es',
                    'WComp-ness', 'WConcavity', 'WConcavPoints', 
                    'WSymm-ry', 'WFractDimens'])
df['Malignant'] = df.Diagnosis.map({'B':0, 'M':1})
df = df.drop('Diagnosis', 1)
df = df.drop('ID', 1)
df['Diagnosis'] = df['Malignant'].copy()
df['Diagnosis'].replace(0, 'Benign',inplace=True)
df['Diagnosis'].replace(1, 'Malignant',inplace=True)
#df.iloc[:5, 29]
df.head()

# Test-Train Split

In [ ]:
from sklearn.cross_validation import train_test_split
X, y = df.iloc[:, :30].values, df.iloc[:, 30].values
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=0)
y_train[:5]

# Feature Scaling 
We can see the features are measured on different scales, and would be beneficial
for almost any classifier. It's gonna be useful for my gradient decent.

   # Standardizaiton

In [ ]:
'''
from sklearn import preprocessing
print(df.iloc[:4,3])

std_scale = preprocessing.StandardScaler().fit( df.iloc[:,2:32].values)
df_std = std_scale.transform( df.iloc[:,2:32].values)
print('Mean after standardization:')
print('   Mean Radius={:.2f}, Mean Area={:.2f}'.format(df_std[:,2].mean(), df_std[:,5].mean()) ,\
      'Mean Smoothness={:.2f} '.format(df_std[:,6].mean()))
print('Stand. dev. after stand-tion:')
print('   Mean Radius={:.2f}, Mean Area={:.2f},'.format(df_std[:,2].std(), df_std[:,5].std()) ,\
      'Mean Smoothness={:.2f} '.format(df_std[:,6].std()))
'''
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Eigendecomposition of the covariance matrix

In [ ]:
'''
cov_mat = np.cov(X_train_std.T)
cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eigh(cov_mat)

print('\nEigenvalues \n%s' % eigen_vals)
'''

# Total and explained variance

In [ ]:
'''
tot = sum(eigen_vals)
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
'''

In [ ]:
'''


plt.bar(range(1, 31), var_exp, alpha=0.5, align='center',
        label='individual variance explained variance')
plt.step(range(1, 31), cum_var_exp, where='mid',
        label='cumulative variance ratio')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.savefig(r'C:\Users\Nursultan\Documents\Data Science\breast cancer\figures\PCA1.png', dpi=300)
plt.tight_layout()
'''

## Feature transformation

In [ ]:
'''
#Make a list of (eigenvalue, eigenvector) tuples
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:,i]) for i in range(len(eigen_vals))]

#Sort the (eigenvalue, eigenvector) tuples from high to low
eigen_pairs.sort(reverse=True)
'''

In [ ]:
'''
w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
              eigen_pairs[1][1][:, np.newaxis],
              #eigen_pairs[2][1][:, np.newaxis]
              ))
w
'''

In [ ]:
#%pylab inline
'''
X_train_pca = X_train_std.dot(w)
colors = ['r', 'g']
markers = ['x', 'o']

for l,c, m in zip(np.unique(y_train), colors, markers):
    plt.scatter(X_train_pca[y_train==l, 0],
                X_train_pca[y_train==l, 1],
                c=c, label=c, marker=m)
plt.xlabel('Principal component 1')
plt.ylabel('Principal component 2')
plt.legend(loc='lower left')
plt.tight_layout()
plt.savefig(r'C:\Users\Nursultan\Documents\Data Science\breast cancer\figures\PCA2.png', dpi=300)



'''

In [ ]:
#X_train_std[0].dot(w)

# Supervised data compression via linear discriminant analysis
        It's a always useful LDA is a dimensionality reduction technique, similar to PCA. The" goal is to
    project a dataset onto a lower-dimensional space with good class-separability in order to avoid overfitting ("curse of dimensionality") and also reduce computational costs." ( From Sebastian Raschka's article) http://sebastianraschka.com/Articles/2014_python_lda.html

# Computing scatter matrices
    Calculate the mean vector for each class

In [ ]:
np.set_printoptions(precision=4)
mean_vecs = []
for label in range(0,2):
    mean_vecs.append(np.mean(X_train_std[y_train==label], axis=0))
    print('MV %s: %s\n' %(label, mean_vecs[label-1]))

Compute the within-class scatter-matrix

In [ ]:
n_features = 30
S_W = np.zeros((n_features, n_features))
for label, mv in zip(range(0,2), mean_vecs):
    class_scatter = np.zeros((n_features, n_features))
    for row in X_train_std[y_train == label]:
        row, mv = row.reshape(n_features, 1), mv.reshape(n_features, 1)
        class_scatter += (row-mv).dot((row-mv).T)
    S_W += class_scatter
print('Within-class scatter matrix: %sx%s ' % (S_W.shape[0], S_W.shape[1]))

In [ ]:
print('Class label distribution: %s' % np.bincount(y_train)[0:])

In [ ]:
n_features = 30
S_W = np.zeros((n_features, n_features))
for label,mv in zip(range(0, 2), mean_vecs):
    class_scatter = np.cov(X_train_std[y_train==label].T)
    S_W += class_scatter
print('Scaled within-class scatter matrix: %sx%s' % (S_W.shape[0], S_W.shape[1]))

Compute the between-class scatter matrix:

In [ ]:
mean_overall = np.mean(X_train_std, axis=0)
n_features = 30
S_B = np.zeros((n_features, n_features))
for i,mean_vec in enumerate(mean_vecs):
    n = X_train[y_train==i+1, :].shape[0]
    mean_vec = mean_vec.reshape(n_features, 1) # make column vector
    mean_overall = mean_overall.reshape(n_features, 1) # make column vector
    S_B += n * (mean_vec - mean_overall).dot((mean_vec - mean_overall).T)

print('Between-class scatter matrix: %sx%s' % (S_B.shape[0], S_B.shape[1]))

# Selecting linear discriminants for the new feature subspace
We are interested in the eigenvectors that correspond to largest eigenvalues

In [ ]:
eigen_vals, eigen_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

#Sorting eigenvectors in decreasing order of the eigenvalues:
# Making a list of (eigenvalue, eigenvector) tuples
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:,i]) for i in range(len(eigen_vals))]

#Soring tuples from high to low
eigen_pairs = sorted(eigen_pairs, key = lambda k : k[0], reverse =True)

print('Eigenvalues in decreasing order:\n')
for eigen_val in eigen_pairs:
    print(eigen_val[0])

In [ ]:
tot = sum(eigen_vals.real)
discr = [(i / tot) for i in sorted(eigen_vals.real, reverse=True)]
cum_discr = np.cumsum(discr)

plt.bar(range(1, 31), discr, alpha=0.5, align='center',
       label='individual "discriminability" ')
plt.step(range(1, 31), cum_discr, where='mid',
        label='cumulative "discriminability"')
plt.ylabel('"discriminability ration')
plt.xlabel('Linear Discriminants')
plt.ylim([-0.1, 1.1])
plt.legend(loc='best')
plt.tight_layout()


In [ ]:
w = np.hstack((eigen_pairs[0][1][:, np.newaxis].real,
                      eigen_pairs[1][1][:, np.newaxis].real))
print('Matrix W:\n', w)

# Projecting onto the new feature space

In [ ]:
X_train_lda = X_train_std.dot(w)
colors = ['r', 'g']
markers = ['x', 'o']

for l, c, m in zip(np.unique(y_train), colors, markers):
    plt.scatter(X_train_lda[y_train==l, 0] * (-1), 
                X_train_lda[y_train==l, 1] * (-1), 
                c=c, label=l, marker=m)

plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower right')
plt.tight_layout()

# LDA via scikit-learn

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train_std, y_train)
X_train_lda.shape


In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.02):
    markers = ('x', 'o', 's', '^')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                         np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=cl)


# Using Gaussian Naive Bayes classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#lr = LogisticRegression()
gnb = GaussianNB()
gnb = gnb.fit(X_train_lda, y_train)
X_test_lda = lda.transform(X_test_std)
y_pred = gnb.predict(X_test_lda)
accuracy_score(y_test, y_pred)

# Using a Feedforward Neural Network
    Setting the model parameters

In [ ]:
num_samples = len(X_train_lda)
print(num_samples)
nn_inputs = 1
nn_outputs = 2
epsilon = 0.01
reg_lambda = 0.01

In [ ]:
def calc_error(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    #z1 = X.dot(W1) + b1
    z1 = X_train_lda.dot(W1) + b1
    #a1= np.tanh(z1)
    #using sigmoid activation function instead of hyperbolic tangent
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores/np.sum(exp_scores, axis=1, keepdims = True)
    
    #calculating the error
    #correct_logprobs = - np.log(probs[range(num_samples), y])
    correct_logprobs = - np.log(probs[range(num_samples), y_train])
    error = np.sum(correct_logprobs)
    #optional regularization term
    error += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_samples * error

In [ ]:
def feed_forward(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    #propagate forward
    #calc the dot product of input layer and its weights and add 1st bias term
    z1 = x.dot(W1) + b1
    #apply tanh activation function
    #a1 = np.tanh(z1)
    #using sigmoid activation function instead of hyperbolic tangent
    a1 = sigmoid(z1)
    #calc the dot product of the output of tanh func applied
    # to input layer and its weights and add 2nd bias term
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores/np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(prob, axis=1)

In [ ]:
#function to learn the param-s for the NN and return the model
# hid_dim - hidden layer dimension - # of nodes in hidden layer
# num_passes - num of passes through the training set for gradient descent
# print_loss - if true, prints the loss every 1000 iterations
def build_model(hid_dim, num_passes = 5000, print_loss=False):
    #initialize param-s randomly, we need to learn them
    np.random.seed(0)
    W1 = np.random.randn(nn_inputs, hid_dim)/ np.sqrt(nn_inputs)
    b1 = np.zeros((1, hid_dim))
    W2 = np.random.randn(hid_dim,nn_outputs) / np.sqrt(hid_dim)
    b2 = np.zeros((1, nn_outputs))
    
    model = {}
    
    #Gradient descent
    for i in xrange(0, num_passes):
        
        #feed forward
        #z1 = X.dot(W1) + b1
        z1 = X_train_lda.dot(W1) + b1
        #a1 = np.tanh(z1)
        #using sigmoid activation function instead of hyperbolic tangent
        a1 = sigmoid(z1)
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        #Backpropagate
        delta3 = probs
        # delta3[range(num_samples), y] -= 1
        delta3[range(num_samples), y_train] -= 1
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        #delta2 = delta3.dot(W2.T) * (1 - np.power(a1,2))
        #derivative of sigmoid function f(x) = 1./ (1+ e^(-x)) is f'(x) = f(x) * (1-f(x))
        delta2 = delta3.dot(W2.T) * (a1 * (1 - a1))
        #dW1 = np.dot(X.T, delta2)
        dW1 = np.dot( X_train_lda.T, delta2)
        db1 = np.sum(delta2, axis =0)
        
        #add regularization terms (b1 and b2 aren't regularized)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1
        
        #Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        #optionally print the loss
        if print_loss and i % 100 == 0:
            print "Loss after iteration %i: %f" %(i, calc_error(model))
    return model

In [ ]:
# Build a model with a 1-dim hidden layer
model = build_model(1, print_loss=True)

# Min-Max Scaling

In [ ]:
from sklearn import preprocessing
minmax_scale = preprocessing.MinMaxScaler().fit(df.iloc[:,:30].values)
df_minmax = minmax_scale.transform(df.iloc[:,:30].values)
print('Min value after min-max:')
print('  Mean Radius={:.2f}, Mean Area={:.2f}'.format(df_minmax[:,0].min(), df_minmax[:,3].min()) ,\
      'Mean Smoothness={:.2f}'.format(df_minmax[:,4].min())) 
print('Max value after min-max:')
print('  Mean Radius={:.2f}, Mean Area={:.2f}'.format(df_minmax[:,0].max(), df_minmax[:,3].max()) ,\
      'Mean Smoothness={:.2f}'.format(df_minmax[:,4].max())) 

# Plotting

In [ ]:



def plot():
    plt.figure(figsize=(8,6))
    
    plt.scatter(df.iloc[:,0], df.iloc[:,4], 
               color = 'green', label='input scale', alpha=0.5)
    plt.scatter(X_train_std[:,0], X_train_std[:,4], color='red',
            label = 'Standardized', alpha=0.3)
    plt.scatter(df_minmax[:,0], df_minmax[:,4], color='blue', 
                label='min-max scaled [min=0, max=1]', alpha=0.3)
    plt.title(" Mean Radius and Mean Smoothness content")
    plt.xlabel('Radius')
    plt.ylabel('Smoothness')
    plt.legend(loc='upper left')
    plt.grid()
    
    plt.tight_layout()

plot()
plt.show()
    

In [ ]:

malignant = df[df['Malignant'].isin([1])]
benign = df[df['Malignant'].isin([0])]
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(malignant['MTexture'], malignant['WArea'], s=50, marker='x', label='Malignant')
ax.scatter(benign['MTexture'], benign['WArea'], s=50, marker='o', label='Benign')
ax.legend()

seabornMatrix = sns.pairplot(df[['Malignant','MTexture','WArea','WSmooth-es']],hue="Malignant")
seabornMatrix.savefig("SeabornMatrix.png")

In [ ]:
#df.index=range(0,569,1)
visualIn3D = plt.figure().gca(projection='3d')
fig3D = plt.gcf()
fig3D.set_size_inches(15.5, 10.5)
visualIn3D.scatter(df['MTexture'], df['WArea'], df['WSmooth-es'], c= 'b')
visualIn3D.set_xlabel('Mean Texture')
visualIn3D.set_ylabel('Worst Area')
visualIn3D.set_zlabel('Worst Smoothness')
plt.show()
fig3D.savefig('3D visualization.png', bbox_inches='tight')